## Pulling Crop Data from BigQuery

- Exploration of pulling crop data from WASDE_MAIN, from google bigquery

#### Import statments

In [15]:
import pandas as pd 
import numpy as np
import google.auth
import google.oauth2.credentials
import pandas_gbq as gbq
import pydata_google_auth
import tqdm

#### Setting up GBQ requirements

In [24]:

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)
gbq.context.credentials = credentials
gbq.context.project = "caleb-database-project-1"

Pulling Data from GBQ

In [7]:
wasde_main = "`caleb-database-project-1.USDA_CLEANED.WASDE_MAIN`"

region_names = gbq.read_gbq("SELECT DISTINCT Region FROM" + wasde_main)
commodty_names = gbq.read_gbq("SELECT DISTINCT Commodity FROM" + wasde_main)
attr_names = gbq.read_gbq("SELECT DISTINCT Attribute FROM" + wasde_main)
projection_names = gbq.read_gbq("SELECT DISTINCT ReliabilityProjection	 FROM" + wasde_main)

Downloading: 100%|██████████| 12/12 [00:00<00:00, 31.83rows/s]


%store helps to store dataframes so constant querying is not required

In [9]:
%store region_names
%store commodty_names
%store attr_names
%store projection_names

Stored 'region_names' (DataFrame)
Stored 'commodty_names' (DataFrame)
Stored 'attr_names' (DataFrame)
Stored 'projection_names' (DataFrame)


run code below to pull saved dataframes

In [4]:
%store -r region_names
%store -r commodty_names
%store -r attr_names
%store -r projection_names

#### Shortlisted countries and respective commodities available



Southeast Asia, Indonesia, Thailand, Vietnam, Burma, Philippines, Australia, India

United States, Argentina, Brazil, Mexico

___

Southeast Asia: Coarse Grain, Corn, Soybean Meal, Wheat, Oilseed, Soybean

Indonesia: Rice, Cotton

Thailand: Rice, Cotton

Vietnam: Rice, Cotton

Burma: Rice

Philippines: Rice

Australia: Coarse Grain, Cotton, Wheat

India: Cotton, Rice, Soybean Meal, Soybean Oil, Wheat
___

United States: 'Coarse Grain', 'Cotton', 'Oilseed, Soybean', 'RICE, milled',
       'Wheat', 'Barley', 'Beef', 'Broiler', 'Corn', 'Eggs',
       'Meal, Soybean', 'Milk', 'Oats', 'Oil, Soybean', 'Pork', 'Sorghum',
       'Turkey', 'Butter', 'Cheese', 'Dry Whey', 'Milk, Class III',
       'Milk, Class IV', 'Nonfat Dry Milk', 'Feed Grains',
       'Red Meat & Poultry', 'Total Poultry', 'Total Red Meat',
       'Dry Whole Milk', 'Milk, Fat Basis', 'Milk, Skim-Soild Basis',
       'Red Meat', 'Barrows and gilts', 'Steers', 'Rice, Long',
       'Rice, Med and Short', 'Rice, Rough', 'Sugar', 'Coarse Grains',
       'Rice, milled', 'Total Grains', 'Oilmeals', 'Oilseeds',
       'Vegetable Oils', 'Rice', 'Soybean Meal', 'Soybean Oil', 'Total',
       'Wheat, Durum', 'Wheat, Hard Red Spring', 'Wheat, Hard Red Winter',
       'Wheat, Soft Red Winter', 'Wheat, White', 'Hard Red Spring',
       'Hard Red Winter', 'Soft Red Winter', 'BEEF', 'BROILER', 'PORK',
       'TOTAL POULTRY', 'TOTAL RED MEAT', 'TURKEY', 'Choice steers'

Argentina: Coarse Grain, Corn, Soybean Meal, Soybean Oil, Oilseed, Soybean, Wheat

Brazil: Coarse Grain, Corn, Cotton, Rice, Soybean Meal, Soybean Oil, Oilseed, Soybean, Wheat

Mexico: Coarse Grain, Corn, Cotton, Rice, Oilseed, Soybean, Sugar, Soybean Meal

Attributes:
Ending Stocks, Exports, Production, Imports, Yield

#### Defining functions for visualisation

In [16]:
import plotly
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

In [30]:
def get_country_data(country):
    '''
    obtain dataframe for specified country
    '''
    df = gbq.read_gbq(
        "select * FROM" + wasde_main +
        "where Region = '" + country + "'"
    ) 
    return df

def get_commodty_attr(country, commodity, attr, ProjEstFlag = None):
    '''
    filter for specified country dataframe, commodity and attribute
    '''

    if ProjEstFlag is None:
            df = country.loc[(country['ProjEstFlag'].isnull()) & (country['Commodity'] == commodity) & (country['Attribute'] == attr) & (country['ReliabilityProjection'].isnull())
                    & (country['Unit'] == 'Million Metric Tons')]
    else:
        df = country.loc[(country['ProjEstFlag'] == ProjEstFlag) & (country['Commodity'] == commodity) & (country['Attribute'] == attr) & (country['ReliabilityProjection'].isnull())
                        & (country['Unit'] == 'Million Metric Tons')]

    # convert ReleaseDate to datetime
    df['ReleaseDate'] = pd.to_datetime(df['ReleaseDate'])

    # set ReleaseDate as the index
    df = df.set_index('ReleaseDate').sort_index()

    # change value to float
    df['Value'] = df.Value.astype(float)

    return df


def plot_values(df):
    '''
    plot filtered commodity data using plotly
    '''
    fig = px.line(df, x = df.index, y = 'Value', hover_data=["MarketYear"])
    fig.update_layout(title = f"{df.Region[0]} {df.Commodity[0]} {df.Attribute[0]} ({df.ProjEstFlag[0]})")
    return fig

Use brazil_df as an example

In [31]:
brazil_df

,WasdeNumber,ReportDate,ReportTitle,Attribute,ReliabilityProjection,Commodity,Region,MarketYear,ProjEstFlag,AnnualQuarterFlag,Value,Unit,ReleaseDate,ReleaseTime,ForecastYear,ForecastMonth
0,512,November 2012,World Coarse Grain Supply and Use,Beginning Stocks,None,Coarse Grain,Brazil,2010/11,None,Annual,10.29,Million Metric Tons,2012-11-09,08:30:00.0000000,2012,11
1,512,November 2012,World Coarse Grain Supply and Use,Domestic Feed,None,Coarse Grain,Brazil,2010/11,None,Annual,45.06,Million Metric Tons,2012-11-09,08:30:00.0000000,2012,11
2,512,November 2012,World Coarse Grain Supply and Use,Domestic Total,None,Coarse Grain,Brazil,2010/11,None,Annual,52.73,Million Metric Tons,2012-11-09,08:30:00.0000000,2012,11
3,512,November 2012,World Coarse Grain Supply and Use,Ending Stocks,None,Coarse Grain,Brazil,2010/11,None,Annual,10.64,Million Metric Tons,2012-11-09,08:30:00.0000000,2012,11
4,512,November 2012,World Coarse Grain Supply and Use,Exports,None,Coarse Grain,Brazil,2010/11,None,Annual,8.42,Million Metric Tons,2012-11-09,08:30:00.0000000,2012,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,511,October 2012,World Wheat Supply and Use,Domestic Total,None,Wheat,Brazil,2012/13,Proj.,Annual,11.00,Million Metric Tons,2012-10-11,08:30:00.0000000,2012,10
23051,511,October 2012,World Wheat Supply and Use,Ending Stocks,None,Wheat,Brazil,2012/13,Proj.,Annual,1.26,Million Metric Tons,2012-10-11,08:30:00.0000000,2012,10
23052,511,October 2012,World Wheat Supply and Use,Exports,None,Wheat,Brazil,2012/13,Proj.,Annual,1.50,Million Metric Tons,2012-10-11,08:30:00.0000000,2012,10
23053,511,October 2012,World Wheat Supply and Use,Imports,None,Wheat,Brazil,2012/13,Proj.,Annual,7.00,Million Metric Tons,2012-10-11,08:30:00.0000000,2012,10


In [1]:
#brazil_df = get_country_data('Brazil')
#%store brazil_df

UsageError: Unknown variable 'brazil_df'


In [2]:
%store -r brazil_df

In [10]:
brazil_df['Attribute'].unique()

array(['Beginning Stocks', 'Domestic Feed', 'Domestic Total',
       'Ending Stocks', 'Exports', 'Imports', 'Production',
       'Domestic Use', 'Loss', 'Total  Domestic', 'Domestic Crush'],
      dtype=object)

In [32]:
country_df_actual_values = get_commodty_attr(brazil_df, 'Corn', 'Production')
country_df_proj_values = get_commodty_attr(brazil_df, 'Corn', 'Production', 'Proj.')

C:\Users\raych\AppData\Local\Temp\ipykernel_11308\3270399950.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\raych\AppData\Local\Temp\ipykernel_11308\3270399950.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
plot_values(country_df_actual_values)

In [34]:
plot_values(country_df_proj_values)